#**Машинное обучение ИБ-2024**

#**Домашнее задание 1.**
#Регрессия, KNN, LinearRegression.

В данной домашней работе мы будем строить модели для предсказания цены квартиры в России. Ниже приведено описание некоторых колонок набора данных.

date - дата публикации объявления

price - цена в рублях

level- этаж, на котором находится квартира

levels - количество этажей в квартире

rooms - количество комнат в квартире. Если значение -1, то квартира считается апартаментами.

area - площадь квартиры.

kitchen_area - площадь кухни.

geo_lat - Latitude

geo_lon - Longitude

building_type - материал застройки. 0 - Don't know. 1 - Other. 2 - Panel. 3 - Monolithic. 4 - Brick. 5 - Blocky. 6 - Wooden

#Часть 0. Начало работы

Для начала работы с данными импортируем библиотеки, которые понадобятся в данном задании.

In [93]:
#!pip3 install scikit-learn numpy matplotlib pandas seaborn --break-system-packages - использовал, чтобы пакеты установить (macos)

import math
import pandas as pd
import numpy as np
import matplotlib as plt
import sklearn
import seaborn as sns

Загрузим библиотеку folium для отображения данных на карте по координатам.

In [94]:
!pip3 install folium --break-system-packages # на macos с поставленным pipx пришлось делать вот так

Загрузим данные из csv файла в датафрейм.

In [95]:
df = pd.read_csv('input_data.csv', sep=";") # читаем данные и указываем separator (;)

Отобразим на карте координаты наших построек.

In [96]:
import folium
from IPython.display import display

map_df = df.loc[:1000]

m = folium.Map(location=[55.751244, 37.618423], zoom_start=10)

lats = map_df['geo_lat'].loc[:1000]
longs = map_df['geo_lon'].loc[:1000]

for point in zip(lats, longs):
    folium.Marker(
        location=[point[0], point[1]]
    ).add_to(m)

display(m)

# Часть 1. Подготовим данные для обработки моделями машинного обучения.

**0.5 Балл**. География наших наблюдений в наборе данных крайне большая. Однако мы знаем, что стоимость квартир в Москве и Санкт-Петербурге намного выше, чем в среднем по России. Давайте сделаем признаки, который показывают, находится ли квартира в 20 килиметрах от центра Москвы или находится ли квартира в 20 килиметрах от центра Санкт-Петербурга.

Создайте два признака is_Moscow и is_Saint_Peterburg. Для нахождения расстояния по координатам используйте функцию haversine_distance.

In [97]:
def distance(lat1, lon1, lat2, lon2):
    # вычисляем расстояние между двумя точками на поверхности земли
    R = 6371
    dlat, dlon = np.radians(lat2 - lat1), np.radians(lon2 - lon1)
    a = np.sin(dlat / 2) ** 2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2) ** 2
    return R * 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

# обозначиваем центры
moscow_center = np.array([55.7522, 37.6156])
spb_center = np.array([59.9386, 30.3141])

# преобразуем столбцы широты и долготы
latitudes = df['geo_lat'].values
longitudes = df['geo_lon'].values

# вычисляем расстояния
dist_to_moscow = distance(latitudes, longitudes, moscow_center[0], moscow_center[1])
dist_to_spb = distance(latitudes, longitudes, spb_center[0], spb_center[1])

# создаём признаки
df['is_Moscow'] = dist_to_moscow <= 20
df['is_Saint_Peterburg'] = dist_to_spb <= 20

**0.5 Балла**. В нашем наборе данных есть признаки, которые мы теоретически можем использовать, например postal_code, но мы это будем делать в рамках домашней работы очень-очень долго. Поэтому предлагается удалить ненужные признаки из датафрейма.

Удалим geo_lat,	geo_lon,	object_type,	postal_code,	street_id,	id_region,	house_id.

In [98]:
df = df.drop(columns=['geo_lat', 'geo_lon', 'object_type', 'postal_code', 'street_id', 'id_region', 'house_id'])

**0.5 Балл**. Для начала Вам предлагается проанализировать Ваши оставшиеся признаки (колонки) в наборе данных. Какие колонки категориальные? Какие числовые?

Категориальные: **building_type (тип здания)**

Числовые: **rooms (количество комнат), price (цена квартиры), level (этаж, на котором находится квартира), levels (количество этажей в здании), area (площадь квартиры), kitchen_area (площадь кухни)**

Бинарные (выделяю отдельно): **is_Moscow, is_SaintPeterburg**

Давайте закодируем категориальные признаки с помощью OneHot-Encoding. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [99]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False, drop='first')

categorical = ['building_type']
encoded_categorical = encoder.fit_transform(df[categorical])

# создание датафрейма с закодированными признаками и удаление старых
encoded_df = pd.DataFrame(encoded_categorical, columns=encoder.get_feature_names_out(categorical))
df = df.drop(columns=categorical)

# объединение оригинального датафрейма с закодированными признаками
df = pd.concat([df, encoded_df], axis=1)

print(df.head())

         date     price  level  levels  rooms  area  kitchen_area  is_Moscow  \
0  2021-01-01   2451300     15      31      1  30.3           0.0      False   
1  2021-01-01   1450000      5       5      1  33.0           6.0      False   
2  2021-01-01  10700000      4      13      3  85.0          12.0      False   
3  2021-01-01   3100000      3       5      3  82.0           9.0      False   
4  2021-01-01   2500000      2       3      1  30.0           9.0      False   

   is_Saint_Peterburg  building_type_1  building_type_2  building_type_3  \
0               False              0.0              0.0              0.0   
1               False              0.0              0.0              0.0   
2               False              0.0              0.0              1.0   
3               False              0.0              0.0              0.0   
4               False              0.0              0.0              1.0   

   building_type_4  building_type_5  building_type_6  
0      

**0.5 Балл**. Поработаем с числовыми признаками:


1.   Добавьте в ваш датасет два признака: количество дней со дня первого наблюдения (разница между датами объявлений). Возможно, для предсказания цены не так важен этаж, как важно отношение этажа квартиры на количество этажей в доме, добавьте этот признак. После добавления нового признака колонку date можно удалить.
2.   Числовые признаки могут иметь разные порядки. Давайте отнормируем числовые признаки с помощью StandartScaller https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html.



In [100]:
from sklearn.preprocessing import StandardScaler

# преобразуем date в формат datetime и считаем количество дней со дня первого наблюдения
df['date'] = pd.to_datetime(df['date'])
df['days_since_first_observation'] = (df['date'] - df['date'].min()).dt.days
df = df.drop(columns=['date'])

# добавляем признак отношения этажа квартиры к количеству этажей в доме
df['level_to_levels_ratio'] = df['level'] / df['levels']

numerical = ['price', 'level', 'levels', 'rooms', 'area', 'kitchen_area', 'days_since_first_observation', 'level_to_levels_ratio']

# фильтруем строки 
df = df[~df['level_to_levels_ratio'].isin([float('inf'), -float('inf')])]

scaler = StandardScaler()
df[numerical] = scaler.fit_transform(df[numerical])

print(df.head())

      price     level    levels     rooms      area  kitchen_area  is_Moscow  \
0 -0.021928  1.622862  2.665124 -0.621444 -0.840594      0.082513      False   
1 -0.026992 -0.270005 -0.937264 -0.621444 -0.741061      0.267578      False   
2  0.019789 -0.459292  0.171164  1.106218  1.175873      0.452643      False   
3 -0.018648 -0.648579 -0.937264  1.106218  1.065280      0.360110      False   
4 -0.021682 -0.837865 -1.214370 -0.621444 -0.851653      0.360110      False   

   is_Saint_Peterburg  building_type_1  building_type_2  building_type_3  \
0               False              0.0              0.0              0.0   
1               False              0.0              0.0              0.0   
2               False              0.0              0.0              1.0   
3               False              0.0              0.0              0.0   
4               False              0.0              0.0              1.0   

   building_type_4  building_type_5  building_type_6  \
0     

**2 Балла**. Реализуйте класс KNNRegressor, который должен делать регрессию методом k ближайших соседей.

In [101]:
from sklearn.metrics import pairwise_distances

class KNNRegressor:
    def __init__(self, n_neighbors=5, metric='euclidean'):
        self.n_neighbors = n_neighbors
        self.metric = metric
        self.X_train = None
        self.y_train = None

    def fit(self, X, y):
        # сохраняем трен. данные
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        # вычисляем расстояния между тестовыми данными и тренировочными
        distances = pairwise_distances(X, self.X_train, metric=self.metric)
        # индексы k ближайших
        neighbors_indices = np.argsort(distances, axis=1)[:, :self.n_neighbors]
        # среднее значение переменной для ближайших соседей
        predictions = np.mean(self.y_train[neighbors_indices], axis=1)
        return predictions

**3 Балла**. Реализуйте класс LinearRegression, поддерживающий обучение градиентными спусками SGD, Momentum, AdaGrad. Используйте градиент для оптимизации функции потерь MSE.

In [102]:
class LinearRegression:
    def __init__(self, learning_rate=0.01, optimization='SGD', epsilon=1e-8, decay_rate=0.9, max_iter=1000):
        self.learning_rate = learning_rate
        self.optimization = optimization
        self.epsilon = epsilon
        self.decay_rate = decay_rate
        self.max_iter = max_iter 
        self.weights = None 
        self.bias = None 

    def fit(self, X, y):
        # веса и смещение
        num_samples, num_features = X.shape
        self.weights = np.zeros(num_features)
        self.bias = 0

        # переменные для оптимизации
        if self.optimization == 'Momentum':
            self.velocity_weights = np.zeros(num_features)
            self.velocity_bias = 0
        elif self.optimization == 'AdaGrad':
            self.gradient_squared_weights = np.zeros(num_features)
            self.gradient_squared_bias = 0

        for _ in range(self.max_iter):
            # предсказания
            y_pred = np.dot(X, self.weights) + self.bias

            error = y_pred - y
            gradient_weights = (1 / num_samples) * np.dot(X.T, error) # по весам
            gradient_bias = (1 / num_samples) * np.sum(error) # по смещению

            if self.optimization == 'SGD':
                self.weights -= self.learning_rate * gradient_weights
                self.bias -= self.learning_rate * gradient_bias

            elif self.optimization == 'Momentum':
                self.velocity_weights = self.decay_rate * self.velocity_weights + (1 - self.decay_rate) * gradient_weights
                self.velocity_bias = self.decay_rate * self.velocity_bias + (1 - self.decay_rate) * gradient_bias
                self.weights -= self.learning_rate * self.velocity_weights
                self.bias -= self.learning_rate * self.velocity_bias

            elif self.optimization == 'AdaGrad':
                self.gradient_squared_weights += gradient_weights ** 2
                self.gradient_squared_bias += gradient_bias ** 2
                adjusted_learning_rate_weights = self.learning_rate / (np.sqrt(self.gradient_squared_weights) + self.epsilon)
                adjusted_learning_rate_bias = self.learning_rate / (np.sqrt(self.gradient_squared_bias) + self.epsilon)
                self.weights -= adjusted_learning_rate_weights * gradient_weights
                self.bias -= adjusted_learning_rate_bias * gradient_bias

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias

# Часть 2. Эксперименты с моделями машинного обучения.

**3 Балла**. Проведите эксперименты с написанными Вами методами машинного обучения. Выделите обучающую и тестовую выборки в отношении 0,8 и 0,2 соответственно. Измерьте ошибку MSE, MAE, RMSE. Заиспользуйте методы KNNRegressor и LinearRegression из библиотеки sklearn, сравните качество Ваших решений и библиотечных.

In [103]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression as SklearnLinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.impute import SimpleImputer
from math import sqrt

def evaluate_model(model, x_train, y_train, x_test, y_test):
    # оценка модели и возврат метрик
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = sqrt(mse)
    return mse, mae, rmse

# выборка
sampled_df = df.sample(frac=0.01, random_state=88)
imputer = SimpleImputer(strategy='mean')

features = imputer.fit_transform(sampled_df.drop(columns=['price']).values)  # признаки
target = sampled_df['price'].fillna(sampled_df['price'].mean()).values  # цель

# разделение данных
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=88)

# KNN (мой)
custom_knn_model = KNNRegressor(n_neighbors=5)
mse_knn_custom, mae_knn_custom, rmse_knn_custom = evaluate_model(custom_knn_model, x_train, y_train, x_test, y_test)

# KNN (sklearn)
sklearn_knn_model = KNeighborsRegressor(n_neighbors=5)
mse_knn_sklearn, mae_knn_sklearn, rmse_knn_sklearn = evaluate_model(sklearn_knn_model, x_train, y_train, x_test, y_test)

# линейная регрессия (моя)
custom_lr_model = LinearRegression(learning_rate=0.01, optimization='SGD', max_iter=1000)
mse_lr_custom, mae_lr_custom, rmse_lr_custom = evaluate_model(custom_lr_model, x_train, y_train, x_test, y_test)

# линейная регрессия (sklearn)
sklearn_lr_model = SklearnLinearRegression()
mse_lr_sklearn, mae_lr_sklearn, rmse_lr_sklearn = evaluate_model(sklearn_lr_model, x_train, y_train, x_test, y_test)

results = [
    ["Мой KNN", mse_knn_custom, mae_knn_custom, rmse_knn_custom],
    ["sklearn KNN", mse_knn_sklearn, mae_knn_sklearn, rmse_knn_sklearn],
    ["Мой LinearRegression", mse_lr_custom, mae_lr_custom, rmse_lr_custom],
    ["sklearn SklearnLinearRegression", mse_lr_sklearn, mae_lr_sklearn, rmse_lr_sklearn],
]

print("\nСводная таблица результатов:")
print(f"{'Модель':<20} {'MSE':<20} {'MAE':<20} {'RMSE':<20}")
for model, mse, mae, rmse in results:
    print(f"{model:<20} {mse:<20.11f} {mae:<20.11f} {rmse:<20.11f}")


Сводная таблица результатов:
Модель               MSE                  MAE                  RMSE                
Мой KNN              0.05780757379        0.01545488002        0.24043205649       
sklearn KNN          0.05780756472        0.01545442067        0.24043203763       
Мой LinearRegression 0.05487365296        0.02030536654        0.23425126031       
sklearn SklearnLinearRegression 0.05477599398        0.02089620092        0.23404271828       
